In [22]:
import tensorflow as tf
import os
import numpy as np
from numpy import unicode
import h5py
import glob
import cv2
from PIL import Image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import keras.backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Lambda, Input, Dense, GlobalAveragePooling2D, Merge, Dropout
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from imutils import build_montages

sess = tf.InteractiveSession()

In [54]:
"""Define functions to create the triplet loss with online triplet mining."""

def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.

    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = tf.matmul(embeddings, tf.transpose(embeddings))

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = tf.diag_part(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm, 1) - 2.0 * dot_product + tf.expand_dims(square_norm, 0)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.maximum(distances, 0.0)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16

        distances = tf.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances


def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.

    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]

    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    # Combine the two masks
    mask = tf.logical_and(indices_not_equal, labels_equal)

    return mask


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.

    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]

    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.logical_not(labels_equal)

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.

    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]

    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    distinct_indices = tf.logical_and(tf.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    # Combine the two masks
    mask = tf.logical_and(distinct_indices, valid_labels)

    return mask


def batch_all_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    We generate all the valid triplets and average the loss over the positive ones.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # shape (batch_size, batch_size, 1)
    anchor_positive_dist = tf.expand_dims(pairwise_dist, 2)
    assert anchor_positive_dist.shape[2] == 1, "{}".format(anchor_positive_dist.shape)
    # shape (batch_size, 1, batch_size)
    anchor_negative_dist = tf.expand_dims(pairwise_dist, 1)
    assert anchor_negative_dist.shape[1] == 1, "{}".format(anchor_negative_dist.shape)

    # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
    # triplet_loss[i, j, k] will contain the triplet loss of anchor=i, positive=j, negative=k
    # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
    # and the 2nd (batch_size, 1, batch_size)
    triplet_loss = anchor_positive_dist - anchor_negative_dist + margin

    # Put to zero the invalid triplets
    # (where label(a) != label(p) or label(n) == label(a) or a == p)
    mask = _get_triplet_mask(labels)
    mask = tf.to_float(mask)
    triplet_loss = tf.multiply(mask, triplet_loss)

    # Remove negative losses (i.e. the easy triplets)
    triplet_loss = tf.maximum(triplet_loss, 0.0)

    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = tf.to_float(tf.greater(triplet_loss, 1e-16))
    num_positive_triplets = tf.reduce_sum(valid_triplets)
    num_valid_triplets = tf.reduce_sum(mask)
    fraction_positive_triplets = num_positive_triplets / (num_valid_triplets + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    triplet_loss = tf.reduce_sum(triplet_loss) / (num_positive_triplets + 1e-16)

    return triplet_loss, fraction_positive_triplets


def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    For each anchor, we get the hardest positive and hardest negative to form a triplet.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)
    #print(pairwise_dist.eval())

#     # For each anchor, get the hardest positive
#     # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.to_float(mask_anchor_positive)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(tf.cast(mask_anchor_positive, tf.float64), pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_positive_dist", tf.reduce_mean(hardest_positive_dist))

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.to_float(mask_anchor_negative)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - tf.cast(mask_anchor_negative, tf.float64))

#     # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_negative_dist", tf.reduce_mean(hardest_negative_dist))

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(triplet_loss)
    #print(triplet_loss.eval())

    return triplet_loss


In [5]:
def get_triplets(data, labels):
    pos_label, neg_label = np.random.choice(labels, 2, replace=False)
    pos_indexes = np.where(labels == pos_label)[0]
    neg_indexes = np.where(labels == neg_label)[0]
    np.random.shuffle(pos_indexes)
    np.random.shuffle(neg_indexes)
    anchor = data[pos_indexes[0]]
    positive = data[pos_indexes[-1]]
    negative = data[neg_indexes[0]]
    return (anchor, positive, negative)

In [6]:
def dump_features(image_paths, labels, hdf5_path, feature_extractor, image_formats=("jpg")):
    # image_paths = []
    # for image_format in image_formats:
    #     image_paths += glob.glob("{}/*.{}".format(images_dir, image_format))
    # image_paths = sorted(image_paths)
    db = h5py.File(hdf5_path, mode="w")
    features_shape = ((len(labels),), feature_extractor.output_shape[1:])
    features_shape = [dim for sublist in features_shape for dim in sublist]
    imageIDDB = db.create_dataset("image_ids", shape=(len(labels),),
                                  dtype=h5py.special_dtype(vlen=unicode))
    featuresDB = db.create_dataset("features",
                                   shape=features_shape, dtype="float")
    labelsDB = db.create_dataset("labels",
                                 shape=(len(labels),), dtype="int")
    for i in range(0, len(labels), 16):
        start,end = i, i+16
        image_ids = [path.split("/")[-1] for path in image_paths[start:end]]
        images = [cv2.imread(path,1) for path in image_paths[start:end]]
        features = feature_extractor.extract(images)
        imageIDDB[start:end] = image_ids
        featuresDB[start:end] = features
        labelsDB[start:end] = labels[start:end]
        print("Extracting {}/{}".format(i+1+16, len(labels)))
    db.close()

In [7]:
def extract_features(hdf5_path):
    db = h5py.File(hdf5_path,mode="r")
    features = db["features"][:]
    labels = db["labels"][:]

    return (features, labels)

def extract_embeddings(features, model):
    embeddings = model.predict([features, features, features])
    return embeddings[:,:,0]

In [8]:
def euclidean_distance(a,b):
    return K.sqrt(K.sum(K.square((a-b)), axis=1))

def triplet_loss(y_true, anchor_positive_negative_tensor):
    anchor = anchor_positive_negative_tensor[:,:,0]
    positive = anchor_positive_negative_tensor[:,:,1]
    negative = anchor_positive_negative_tensor[:,:,2]
    Dp = euclidean_distance(anchor, positive)
    Dn = euclidean_distance(anchor, negative)

    return K.maximum(0.0, 1+K.mean(Dp-Dn))

In [9]:
class ImageNetFeatureExtractor(object):
    def __init__(self, model="vgg16", resize_to=(150, 150)):
        # MODEL_DICT = {"vgg16": VGG16, "vgg19": VGG19, "inception": InceptionV3, "resnet": ResNet50,
        #               "xception": Xception}
        # network = MODEL_DICT[model.lower()]
        self.model_name = model.lower()
        self.model = self.getModel()
        self.preprocess_input = preprocess_input
        self.imageSize = resize_to

    def extract(self, images):
        images = self.preprocess(images)
        return self.model.predict(images)

    def getModel(self):
        model = load_model('C:/Users/hp/Downloads/data/model/top_model_vgg/final_model.h5')
        intermediate_layer_model = Model(inputs=model.input,
                                         outputs=model.get_layer("block5_pool").output)

        return intermediate_layer_model

    @property
    def output_shape(self):
        return self.model.compute_output_shape([[None, self.imageSize[0], self.imageSize[1], 3]])

    def resize_images(self, images):
        images = np.array([cv2.resize(image, (self.imageSize[0], self.imageSize[1])) for image in images])
        return images

    def preprocess(self, images):
        images = self.resize_images(images)
        images = self.preprocess_input(images.astype("float"))
        return images

def concat_tensors(tensors, axis=-1):
    return K.concatenate([K.expand_dims(t, axis=axis) for t in tensors])


def get_small_network(input_shape=(None, 4, 4, 512)):
    model = Sequential()
    model.add(GlobalAveragePooling2D(input_shape=input_shape[1:]))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation="relu"))
    #model.add(Dense(128, activation="relu"))
    return model

def get_triplet_network(input_shape=(None, 4, 4, 512)):
    base_model = get_small_network(input_shape=input_shape)

    anchor_input = Input(input_shape[1:])
    positive_input = Input(input_shape[1:])
    negative_input = Input(input_shape[1:])

    anchor_embeddings = base_model(anchor_input)
    positive_embeddings = base_model(positive_input)
    negative_embeddings = base_model(negative_input)

    output = Lambda(concat_tensors)([anchor_embeddings, positive_embeddings, negative_embeddings])
    model = Model([anchor_input, positive_input, negative_input], output)

    return model

In [ ]:
image_path_list=[]
labels=[]
train_data_dir = 'C:/Users/hp/Downloads/data/train/'
cat_root = train_data_dir + 'cats' + '/'
cats=os.listdir(cat_root)

for cat in cats:
    image_path_list.append(cat_root + cat)
    labels.append(0)

dog_root = train_data_dir + 'dogs' + '/'
dogs=os.listdir(dog_root)
for dog in dogs:
    image_path_list.append(dog_root + dog)
    labels.append(1)

labels = np.array(labels)

In [56]:
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')
model = get_triplet_network(features.shape)
nSamples, nWidth, nHeight, nVolume = features.shape
embeddings = features.reshape(nSamples, nWidth * nHeight * nVolume)
loss = batch_hard_triplet_loss(labels, embeddings, margin=1,
                                       squared=True)

In [19]:
features.T.shape

(512, 4, 4, 1200)

In [20]:
embeddings.shape

(1200, 8192)

In [57]:
# data = []
# for i in range(len(features)):
#     anchor, positive, negative = get_triplets(features, labels)
#     data.append([anchor, positive, negative])
# data = np.array(data)
model_check_point_loc = 'C:/Users/hp/Downloads/data/similarity/vgg16_cats_dogs_dup.h5'
targets = np.zeros(shape=(1200, 256, 3))
callback = ModelCheckpoint(model_check_point_loc, period=1, monitor="val_loss")
#X_train, X_test, Y_train, Y_test = train_test_split(data, targets)
model.compile(optimizers.Adam(1e-4), loss)
# model.fit([X_train[:,0], X_train[:,1], X_train[:,2]], Y_train, epochs=10,
#           validation_data=([X_test[:,0], X_test[:,1], X_test[:,2]], Y_test),
#           callbacks=[callback], batch_size=16)

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [ ]:
model.fit([X_train[:,0], X_train[:,1], X_train[:,2]], Y_train, epochs=10,
          validation_data=([X_test[:,0], X_test[:,1], X_test[:,2]], Y_test),
          callbacks=[callback], batch_size=16)

In [34]:
def calculateDistance(embeddings, squared=False):
    dot_product = tf.matmul(embeddings, tf.transpose(embeddings))
    square_norm = tf.diag_part(dot_product)
    print(dot_product.eval())
    print('##############')
    print(square_norm.eval())

In [35]:
matrix = np.random.randint(16, size = (4, 3))
print(matrix)

calculateDistance(matrix)

tf.reset_default_graph()
sess.close()

[[ 9 15 11]
 [ 4  8  5]
 [13  4 13]
 [ 5  9 12]]
[[427 211 320 312]
 [211 105 149 152]
 [320 149 354 257]
 [312 152 257 250]]
##############
[427 105 354 250]


In [50]:
embeddings = np.random.randint(16, size = (3, 2))
print(matrix)

dot_product = tf.matmul(embeddings, K.transpose(embeddings))
print(dot_product.eval())

# Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
# This also provides more numerical stability (the diagonal of the result will be exactly 0).
# shape (batch_size,)
square_norm = tf.diag_part(dot_product)
print(square_norm.eval())

# Compute the pairwise distance matrix as we have:
# ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
# shape (batch_size, batch_size)
distances = tf.expand_dims(square_norm, 1) - 2 * dot_product + tf.expand_dims(square_norm, 0)
print(distances.eval())

[[ 9 15 11]
 [ 4  8  5]
 [13  4 13]
 [ 5  9 12]]
[[338 195 221]
 [195 153 132]
 [221 132 145]]
[338 153 145]
[[  0 101  41]
 [101   0  34]
 [ 41  34   0]]


In [36]:
K.expand_dims([1, 4, 5], 1)

<tf.Tensor 'ExpandDims:0' shape=(3, 1) dtype=int32>

In [38]:
K.expand_dims([1, 4, 5], 0)

<tf.Tensor 'ExpandDims_1:0' shape=(1, 3) dtype=int32>